In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 6, Finished, Available)

# Set up data configuration

In [2]:
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 7, Finished, Available)

# Read the whole Reddit data

In [3]:
comments_path = "reddit-parquet/comments/"
submissions_path = "reddit-parquet/submissions/"

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 8, Finished, Available)

In [4]:
comments_df = spark.read.parquet(f"{wasbs_base_url}{comments_path}")
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 9, Finished, Available)

# filter the subreddit that only contains popular game titles

In [6]:
from pyspark.sql.functions import col, asc,desc

submissions_filtered = submissions_df.filter(col("subreddit").isin(["Minecraft", "Eldenring","Genshin_Impact","pokemongo","RocketLeagueExchange","VALORANT","DestinyTheGame","CallOfDutyMobile","FIFA","pokemon"]))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 11, Finished, Available)

In [7]:
submissions_pop_games = submissions_filtered.groupBy("subreddit").count().orderBy(col("count"), ascending=False).collect()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 12, Finished, Available)

In [8]:
submissions_pop_games_df = spark.createDataFrame(submissions_pop_games).toPandas()
submissions_pop_games_df

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 13, Finished, Available)

,subreddit,count
0,Minecraft,903622
1,Eldenring,827345
2,Genshin_Impact,676233
3,pokemongo,417475
4,RocketLeagueExchange,415833
5,VALORANT,391354
6,DestinyTheGame,379109
7,CallOfDutyMobile,366513
8,FIFA,359291
9,pokemon,355231


In [9]:
import os
CSV_DIR = os.path.join("Users/ky285/fall-2023-reddit-project-team-11/data", "csv")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 14, Finished, Available)

In [11]:
submissions_pop_games_df.to_csv(f"{CSV_DIR}/top_10_pop_games.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 0, 16, Finished, Available)

# show the popularity of each popular game each month